In [2]:
from utils import setup_env_path

setup_env_path()

Former working directory:  c:\Users\corne\Documents\2-Scolarite\1-AgroParisTech\4-Cursus\3-3A\5-Cours\14-Fil_Rouge\fil-rouge-pollinisateurs\notebooks
Current working directory:  c:\Users\corne\Documents\2-Scolarite\1-AgroParisTech\4-Cursus\3-3A\5-Cours\14-Fil_Rouge\fil-rouge-pollinisateurs


In [3]:
# Chargement des librairies
import pandas as pd
import data_quality.insect_treatment.insect_classification_functions as classification_functions

C:\Users\corne\AppData\Local\Temp\ipykernel_12260\3249413620.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Préparation des données 

L'objectif est d'obtenir un dataset avec uniquement les colonnes insecte_sc et insecte_fr.  

De plus, toutes les lignes où il y a soit NA, soit "je ne sais pas", soit "autres" dans ces deux colonnes seront supprimées.

Une attention particulière ser apportée aux lignes possédant une valeur dans leur colonne insecte_fr mais pas dans insecte_sc. En effet, une analyse préliminaire a montré que ce cas existe pour exactement 12 valeurs de insecte_fr, qui sont :
- "Les Mouches difficiles à déterminer",
- "Les Coléoptères difficiles à déterminer",
- "Les Moustiques, Tipules et autres diptères Nématocères",
- "Les Chenilles et fausses-Chenilles",
- "Les Syrphes difficiles à déterminer",
- "Les Terebrants Chalcidiens et autres",
- "Les Epeires et autres Araneidae",
- "Les Punaises difficiles à déterminer",
- "Les Syrphes aux fémurs enflés",
- "Les Tachinaires difficiles à déterminer",
- "Les Punaises prédatrices ternes".  

Cette liste comprend des termes englobant des groupes assez larges d'insectes. Nous allons donc leur attribuer une valeur de insecte_sc qui pourra au mieux les englober d'après nos recherches et connaissances. Les *Punaises prédatrices ternes* n'ayant pas été identifié dans la littérature, nous les remplacerons par le nom scientifique des *Punaises ternes*.
- "Diptera", 
- "Coleoptera", 
- "Nematocera",
- "Lepidoptera", 
- "Syrphinae", 
- "Chalcidoidea",
- "Araneidae", 
- "Heteroptera", 
- "Syrphinae",
- "Tachinidae",
- "Lygus lineolaris".

Nous sélectionnerons les lignes où ces pairs insecte_sc/insecte_fr sont uniques.

In [4]:
# Import spipoll.csv as a pandas dataframe
spipoll = pd.read_csv("data/spipoll.csv", low_memory=False)

# Extract relevant columns from the spipoll dataframe
insect = spipoll[['insecte_sc', 'insecte_fr']]
insect.shape # (670744, 2)

(670744, 2)

In [5]:
values_to_check = ["Je ne sais pas", "Insecte inconnu"]

# Suppression des lignes ayant NA ou les values_to_check dans insecte_sc et dans insecte_fr
insect_no_info = insect.loc[
    (
        (insect["insecte_sc"].isna() | insect["insecte_sc"].isin(values_to_check)) &
        (insect["insecte_fr"].isna() | insect["insecte_fr"].isin(values_to_check))
    )
]
insect_no_info.shape # (11667, 2)

(11667, 2)

In [6]:
insect_no_info.head(30) # la sélection s'est bien passée si on retrouve des NA, "Je ne sais pas" et "Insecte inconnu"

,insecte_sc,insecte_fr
10,Insecte inconnu,Insecte inconnu
21,Insecte inconnu,Insecte inconnu
35,Insecte inconnu,Insecte inconnu
45,Insecte inconnu,Insecte inconnu
53,Insecte inconnu,Insecte inconnu
89,Insecte inconnu,Insecte inconnu
106,Insecte inconnu,Insecte inconnu
126,Insecte inconnu,Insecte inconnu
147,Insecte inconnu,Insecte inconnu
278,Insecte inconnu,Insecte inconnu


In [7]:
# Get the indices of the rows to be removed
indices_to_remove = insect_no_info.index

# Remove the rows
insect = insect.drop(indices_to_remove)
insect.shape # (659077, 2)

(659077, 2)

In [8]:
# Duplicates deletion
insect = insect.drop_duplicates()
insect.shape # (577, 2)

(577, 2)

In [9]:
# Complete the insecte_sc column with the insecte_fr column information
original = ["Les Mouches difficiles à déterminer",
            "Les Coléoptères difficiles à déterminer",
            "Les Moustiques, Tipules et autres diptères Nématocères",
            "Les Chenilles et fausses-Chenilles",
            "Les Syrphes difficiles à déterminer",
            "Les Terebrants Chalcidiens et autres",
            "Les Epeires et autres Araneidae",
            "Les Punaises difficiles à déterminer",
            "Les Syrphes aux fémurs enflés",
            "Les Tachinaires difficiles à déterminer",
            "Les Punaises prédatrices ternes"]
replacement = ["Diptera", "Coleoptera", "Nematocera",
               "Lepidoptera", "Syrphinae", "Chalcidoidea",
               "Araneidae", "Heteroptera", "Syrphinae",
               "Tachinidae", "Lygus lineolaris"]

def replace_insecte_sc(row):
    if pd.isna(row["insecte_sc"]) and row["insecte_fr"] in original:
        index = original.index(row["insecte_fr"])
        return replacement[index]
    else:
        return row["insecte_sc"]

insect["insecte_sc"] = insect.apply(replace_insecte_sc, axis=1)
insect.shape # (577, 2)

(577, 2)

In [10]:
insect.head(10) # la sélection s'est bien passée si on retrouve des valeurs remplacées

,insecte_sc,insecte_fr
0,Apis mellifera,L'Abeille mellifère
1,Bombus,Les Bourdons noirs à bande(s) jaune(s) et cul ...
3,Sarcophaga,Les Mouches à damier
5,Tettigonoidea,Les Sauterelles
6,"Neomyia, Calliphora et autres",Les Mouches aux reflets métalliques
7,Diptera,Les Mouches difficiles à déterminer
13,Tephritidae et autres,Les Téphritides et autres
16,"Eupeodes, Scaeva",Les Syrphes à taches en virgules
18,Episyrphus balteatus,Le Syrphe ceinturé
19,Meliscaeva et autres,Les Syrphes à abdomen fin


In [11]:
# Verification that the replacement happened on the whole dataset
insect['insecte_sc'].isna().sum() # 0

0

In [12]:
# If insecte_sc is "autres", delete the value of insect_sc and keep the value of insect_fr
for i in range(len(insect)):
    if insect.iloc[i, 0] == "autres":
        insect.iloc[i, 0] = None

In [13]:
# Print the value of insecte_fr if insecte_sc is None
insect[insect["insecte_sc"].isna()]

,insecte_sc,insecte_fr
133,None,Les Araignées
891,None,Les Osmies
1701,None,Les Tenthrèdes
7989,None,Les Taupins


In [14]:
original = ["Les Araignées", "Les Osmies", "Les Tenthrèdes", "Les Taupins"]
replacement = ["Araneae", "Osmia", "Tenthredinidae", "Elateridae"]

def replace_insecte_sc(row):
    if pd.isna(row["insecte_sc"]) and row["insecte_fr"] in original:
        index = original.index(row["insecte_fr"])
        return replacement[index]
    else:
        return row["insecte_sc"]

insect["insecte_sc"] = insect.apply(replace_insecte_sc, axis=1)
insect.shape # (577, 2)

(577, 2)

In [15]:
# Ortograph correction
insect.replace("Tettigonoidea", "Tettigonioidea", inplace=True)

In [16]:
# Save the insect dataframe to a csv file
insect.to_csv("data/insect/insect.csv", index=False)

# Classification des insectes

La classification ci-dessous est réalisée à l'aide de la **base de donnée de ITIS**. ITIS (Integrated Taxonomic Information System) est un programme dont l'objectif est de rassembler des données taxonomiques sur les plantes, les animaux, les champignons et autres micro-organismes.

Elle n'a **pas renvoyé de résultats assez satisfaisants** : sur les 519 valeurs uniques existantes dans la colonne *insecte_sc* du dataset spipoll, seules 272 étaient identifiées. On ne pouvait donc pas obtenir la classification d'une moitié d'entre elles environ.

Une autre base de données a donc été utilisée pour réaliser la classification : celle du **NCBI**.

Ce notebook sera conservé pour pour les pré-traitements apportés au dataset initial nous permettant d'obtenir le document *insect.csv*. Il sera réutilisé dans le prochain notebook *insect_classification_ncbi.ipynb*.

In [ ]:
# Chargement du jeu de données
df = pd.read_csv('data/insect/insect.csv')

In [ ]:
# Identification des espèces non présentes dans les databases de pytaxize
unknown = classification_functions.find_unknown(df, 'insecte_sc')

In [ ]:
# Enregistrement des espèces inconnues
unknown_df = pd.DataFrame(unknown, columns=['Unknown'])
unknown_df.to_csv('data/insect/insects_unknown.csv', index=False)

In [ ]:
unknown = pd.read_csv('data/insect/insects_unknown.csv')
unknown = unknown['Unknown'].tolist()
unknown[:10]

In [ ]:
# Suppression des lignes contenant des noms non retrouvés 
df = df[~df['insecte_sc'].isin(unknown)]
len(df) # 309

In [ ]:
# Remplacement de certaines valeurs qui bloquent
# Après avoir fait tourné le bloc ci-dessous
df = df.replace('Stelis, Heriades', 'Heriades')
df = df.replace('Ammophila, Hoplammophila', 'Hoplammophila')
df = df.replace('Allantus, Macrophya et autres', 'Allantus')
df = df.replace('Tibellus, Paratibellus', 'Tibellus')
df = df.replace('Tenthredo, Rhogogaster', 'Tenthredo')
df = df.replace('Nomioides, Celhalictus', 'Nomioides')

In [ ]:
len(df['insecte_sc'].unique()) # 275    

In [ ]:
classif = classification_functions.get_classification_df(df, 'insecte_sc')

In [ ]:
# Création des colonnes Ordre, Super famille, Famille, Sous famille, Genre, Espèce
df['classification'] = df['insecte_sc'].apply(lambda x: classif[x] if x in classif else None)

df['Ordre'] = df['classification'].apply(lambda x: x['Ordre'] if isinstance(x, dict) and 'Ordre' in x else None)
df['Super famille'] = df['classification'].apply(lambda x: x['Super famille'] if isinstance(x, dict) and 'Super famille' in x else None)
df['Famille'] = df['classification'].apply(lambda x: x['Famille'] if isinstance(x, dict) and 'Famille' in x else None)
df['Sous famille'] = df['classification'].apply(lambda x: x['Sous famille'] if isinstance(x, dict) and 'Sous famille' in x else None)
df['Genre'] = df['classification'].apply(lambda x: x['Genre'] if isinstance(x, dict) and 'Genre' in x else None)
df['Espèce'] = df['classification'].apply(lambda x: x['Espèce'] if isinstance(x, dict) and 'Espèce' in x else None)

df.head(30)

In [ ]:
# Enregistrement du nouveau dataframe
df = df.rename(columns={'x': 'Insecte'})
df_classif = df[['Insecte', 'Ordre', 'Super famille', 'Famille', 'Sous famille', 'Genre', 'Espèce']]
df_classif.to_csv('data_insects/insect_classification_without_pp.csv', index=False)
df_classif.head()